# Exercises with Spark

## ZHANG Xin

## First steps with Spark

### 2.1 First RDD

In [ ]:
#2.1
import numpy as np

rddr = np.arange(0, 3000, 1)
rdd21 = sc.parallelize(rddr)

### 2.2 Computing the Sum of Cubes

In [2]:
#2.2
rddr22 = np.power(rddr,3)
rdd22 = sc.parallelize(rddr22)
print("The sum of elements in C is " + str(rdd22.sum()))

The sum of elements in C is 20236502250000

### 2.3 Last digits of elements in $C$

In [3]:
#2.3
rdd23 = rdd22.map(lambda x : (x%10, 1)).reduceByKey(lambda a, b : a+b)
rdd23.sortByKey().collect()

Out[ 41 ]: [(0, 300),
 (1, 300),
 (2, 300),
 (3, 300),
 (4, 300),
 (5, 300),
 (6, 300),
 (7, 300),
 (8, 300),
 (9, 300)]

### 2.4 Digits of $C$ 

In [4]:
#2.4
def digits(i):
    return [e for e in str(i)]
rdd24 = rdd22.flatMap(lambda x : digits(x)).map(lambda x : (x, 1)).reduceByKey(lambda a, b : a+b)
rdd24.sortByKey().collect()

Out[ 42 ]: [('0', 3127),
 ('1', 3667),
 ('2', 3294),
 ('3', 2814),
 ('4', 2762),
 ('5', 2653),
 ('6', 2713),
 ('7', 2787),
 ('8', 2639),
 ('9', 2521)]

## Approximating $\pi$

### 3.1 Step 1 : computing set of pairs

In [5]:
#3.1
rdd31 = rdd21.cartesian(rdd21)
rdd31.max()
rdd31.count()

Out[ 46 ]: 9000000

### 3.2 Step 2 : computing the pairs

In [6]:
#3.2
rdd32 = rdd31.filter(lambda x : (2*x[0]+1)**2+(2*x[1]+1)**2<6000**2)
rdd32.count()

Out[ 48 ]: 7068585

### 3.3 Computing the approximation

In [7]:
#3.3
print("$\pi$: " + str(4*(rdd32.count()/rdd31.count())))

$\pi$: 3.1415933333333332

## Using the Movie Lens dataset

### 4.1 Getting the data

In [8]:
#4.1
#Upload the data

### 4.2 Getting the dataset into an RDD

In [9]:
#4.2
import re
future_pattern = re.compile("""([^,"]+|"[^"]+")(?=,|$)""")
def parseCSV(line):
    return future_pattern.findall(line)
path_data = "/FileStore/tables/"
ratingsFile = sc.textFile(path_data+"/ratings.csv").map(parseCSV)
moviesFile = sc.textFile(path_data+"/movies.csv").map(parseCSV)
moviesFile.take(2)

Out[ 65 ]: [['movieId', 'title', 'genres'],
 ['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy']]

In [10]:
#4.2
ratingsFile.take(2)

Out[ 67 ]: [['userId', 'movieId', 'rating', 'timestamp'], ['1', '1', '4.0', '964982703']]

### 4.3 Cleaning data

In [11]:
#4.3
header = ratingsFile.first() #extract header
ratings = ratingsFile.filter(lambda row : row != header)   #filter out header
ratings.take(2)

Out[ 68 ]: [['1', '1', '4.0', '964982703'], ['1', '3', '4.0', '964981247']]

In [12]:
mHeader = moviesFile.first() #extract header
movies = moviesFile.filter(lambda row : row != mHeader)   #filter out header
movies.take(2)

Out[ 69 ]: [['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'],
 ['2', 'Jumanji (1995)', 'Adventure|Children|Fantasy']]

In [13]:
#cast the third column
ratings = ratings.map(lambda x : [x[0],x[1],float(x[2]),x[3]])
ratings.take(10)

Out[ 78 ]: [['1', '1', 4.0, '964982703'],
 ['1', '3', 4.0, '964981247'],
 ['1', '6', 4.0, '964982224'],
 ['1', '47', 5.0, '964983815'],
 ['1', '50', 5.0, '964982931'],
 ['1', '70', 3.0, '964982400'],
 ['1', '101', 5.0, '964980868'],
 ['1', '110', 4.0, '964982176'],
 ['1', '151', 5.0, '964984041'],
 ['1', '157', 5.0, '964984100']]

### 4.4 10 best movies of all times

In [14]:
#4.4
rankRating = ratings.map(lambda x : (x[1],[x[2],1])).reduceByKey( lambda x, y : (x[0] + y[0], x[1] + y[1])).map(lambda x : [x[0],x[1][0]/x[1][1]])
rankRating.sortBy(lambda x : -x[1]).take(10)

Out[ 86 ]: [['158882', 5.0],
 ['60737', 5.0],
 ['27704', 5.0],
 ['175431', 5.0],
 ['25906', 5.0],
 ['136834', 5.0],
 ['104780', 5.0],
 ['134095', 5.0],
 ['1310', 5.0],
 ['72692', 5.0]]

### 4.5 Ordered list of movies with names

In [15]:
#4.5
rankj = rankRating.map(lambda x : (x[0],x[1]))
moviesj = movies.map(lambda x : (x[0],x[1]))
joined = rankj.join(moviesj).map(lambda x : (x[1][1],x[1][0]))
joined.sortBy(lambda x : -x[1]).take(10)

Out[ 95 ]: [('All Yours (2016)', 5.0),
 ('Watching the Detectives (2007)', 5.0),
 ('Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)', 5.0),
 ('Presto (2008)', 5.0),
 ('Into the Abyss (2011)', 5.0),
 ('Black Tar Heroin: The Dark End of the Street (2000)', 5.0),
 ('Nasu: Summer in Andalusia (2003)', 5.0),
 ('Human (2015)', 5.0),
 ('Tokyo Tribe (2014)', 5.0),
 ('Hollywood Chainsaw Hookers (1988)', 5.0)]

### 4.6 Better ordered list

In [16]:
#4.6
#first formula
newRating1 = ratings.map(lambda x : (x[1],[x[2],1])).reduceByKey( lambda x, y : (x[0] + y[0], x[1] + y[1])).map(lambda x : [x[0],x[1][0]/(x[1][1]+1)])
newRating1.sortBy(lambda x : -x[1]).take(10)

Out[ 96 ]: [['318', 4.415094339622642],
 ['858', 4.266839378238342],
 ['1104', 4.261904761904762],
 ['2959', 4.2534246575342465],
 ['1221', 4.226923076923077],
 ['750', 4.224489795918367],
 ['177593', 4.222222222222222],
 ['50', 4.217073170731707],
 ['1213', 4.216535433070866],
 ['260', 4.214285714285714]]

In [17]:
rankj1 = newRating1.map(lambda x : (x[0],x[1]))
newJoined1 = rankj1.join(moviesj).map(lambda x : (x[1][1],x[1][0]))
newJoined1.sortBy(lambda x : -x[1]).take(10)

Out[ 97 ]: [('"Shawshank Redemption, The (1994)"', 4.415094339622642),
 ('"Godfather, The (1972)"', 4.266839378238342),
 ('"Streetcar Named Desire, A (1951)"', 4.261904761904762),
 ('Fight Club (1999)', 4.2534246575342465),
 ('"Godfather: Part II, The (1974)"', 4.226923076923077),
 ('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
 4.224489795918367),
 ('"Three Billboards Outside Ebbing, Missouri (2017)"', 4.222222222222222),
 ('"Usual Suspects, The (1995)"', 4.217073170731707),
 ('Goodfellas (1990)', 4.216535433070866),
 ('Star Wars: Episode IV - A New Hope (1977)', 4.214285714285714)]

In [18]:
#Second formula
import math
newRating2 = ratings.map(lambda x : (x[1],[x[2],1])).reduceByKey( lambda x, y : (x[0] + y[0], x[1] + y[1])).map(lambda x : [x[0],x[1][0]/(x[1][1])*math.log(x[1][1])])
newRating2.sortBy(lambda x : -x[1]).take(10)
rankj2 = newRating2.map(lambda x : (x[0],x[1]))
newJoined2 = rankj2.join(moviesj).map(lambda x : (x[1][1],x[1][0]))
newJoined2.sortBy(lambda x : -x[1]).take(10)

Out[ 99 ]: [('"Shawshank Redemption, The (1994)"', 25.506303124680446),
 ('Forrest Gump (1994)', 24.135559630846686),
 ('Pulp Fiction (1994)', 24.035971735394476),
 ('"Matrix, The (1999)"', 23.593497870526754),
 ('"Silence of the Lambs, The (1991)"', 23.43310709209536),
 ('Star Wars: Episode IV - A New Hope (1977)', 23.37860964684444),
 ('Fight Club (1999)', 23.007601610036865),
 ("Schindler's List (1993)", 22.788076383338726),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)', 22.561506207236786),
 ('"Godfather, The (1972)"', 22.549726244087907)]